# Starts

In [2]:
import pandas as pd
from src.paths import TRAINING_DATA

starts_table = pd.read_parquet(TRAINING_DATA/"integers/start_june.parquet")

## Train-test split

In [4]:
from datetime import datetime
from src.data_splitting import train_test_split

x_train, y_train, x_test, y_test = train_test_split(
    data = starts_table,
    scenario = "start",
    cutoff_date = datetime(2023,11,1,0,0,0),
    target_column = "trips_next_hour"
)

print(f"{x_train.shape=}")
print(f"{y_train.shape=}")
print(f"{x_test.shape=}")
print(f"{y_test.shape=}")

x_train.shape=(1642871, 674)
y_train.shape=(1642871,)
x_test.shape=(981456, 674)
y_test.shape=(981456,)


## XGBoost 

In [5]:
import xgboost as xgb 

In [6]:
trip_columns = [column for column in x_train.columns if column.startswith("trips_")]
x_train_numeric = x_train[trip_columns]

In [7]:
model = xgb.XGBRegressor()
model.fit(x_train_numeric, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [8]:
x_test_numeric = x_test[trip_columns]

predictions = model.predict(x_test_numeric)
predictions

array([0.00072666, 0.00072666, 0.00209589, ..., 0.00072666, 0.00072666,
       0.00072666], dtype=float32)

In [11]:
pd.Series(predictions).value_counts()

0.000727    632356
0.003403      6076
0.005028      5983
0.003778      5539
0.004350      5355
             ...  
0.094444         1
0.054237         1
0.026313         1
0.027654         1
0.019203         1
Name: count, Length: 127439, dtype: int64

In [12]:
from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)

test_mae

0.011622694